<a href="https://colab.research.google.com/github/ThaissaTeodoro/chatgpt_com_whisper_e_python/blob/main/chatgpt_voice_whisper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
language = 'pt'

In [27]:
# https://gist.github.com/korakot/c21c3476c024ad6d56d5f48b0bca92be
from IPython.display import Audio, display, Javascript
from google.colab import output
from base64 import b64decode

In [28]:
message_start = "Olá, Como o ChatGPT pode te ajudar?"
gtts_object = gTTS(text=message_start, lang=language, slow=False)
audio = "/content/dio/whisper/audio/start.wav"
gtts_object.save(audio)
display(Audio(audio, autoplay=True))


In [29]:

RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""

def record(sec=10):
  display(Javascript(RECORD))
  result = output.eval_js('record(%d)' % (sec*1000))
  audio = b64decode(result.split(',')[1])
  with open('/content/dio/whisper/audio/audio.wav','wb') as f:
    f.write(audio)
  return '/content/dio/whisper/audio/audio.wav'  # or webm ?

print('gravando')
record_file = record()
display(Audio(record_file, autoplay=True))


gravando


<IPython.core.display.Javascript object>

In [31]:
!pip install git+https://github.com/openai/whisper.git -q

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [32]:
import whisper

model = whisper.load_model("small")
result = model.transcribe(record_file, fp16=False, language=language)
transcription = result["text"]
print(transcription)


 Quem declarou a independência do Brasil?


In [16]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 934.2 kB/s eta 0:00:00


In [33]:
import openai

openai.api_key = ''

#criar interação com o chat
response = openai.ChatCompletion.create(
    model= "gpt-3.5-turbo",
    messages = [{"role": "user", "content": transcription}]
)
#resposta do chat
chatgpt_response = response.choices[0].message.content
print(chatgpt_response)

A independência do Brasil foi declarada por Dom Pedro I em 7 de setembro de 1822.


In [24]:
!pip install gTTS

In [34]:
from gtts import gTTS
#sintetizar em voz
gtts_object = gTTS(text=chatgpt_response, lang=language, slow=False)
response_audio = "/content/dio/whisper/audio/response_chat.wav"
gtts_object.save(response_audio)
display(Audio(response_audio, autoplay=True))